In [ ]:
import json
import numpy as np
import os
import pandas as pd

from tqdm import tqdm

out_dir = "../../output/type"
os.makedirs(out_dir, exist_ok=True)


In [ ]:
airport = "ksea"
base_dir = '../../datasets/amelia'
traj_version = 'a10v08'


traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')


traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

In [ ]:
agent_types = {}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_IDs = np.unique(data.ID)
    for ID in unique_IDs:
        atype = data[data.ID == ID].Type.astype(int)
        atype = atype[np.diff(atype, prepend=np.nan).astype(bool)].astype(str).tolist()
        key = ''.join(atype)

        if agent_types.get(key) is None:
            agent_types[key] = 0
            key_p = f"{key}_perc"
            agent_types[key_p] = 0.0

        agent_types[key] += 1
        total_count += 1


In [ ]:

for k, v in agent_types.items():
    if "perc" in k: continue
    kp = f"{k}_perc"
    agent_types[kp] = round(v / total_count, 3)

agent_types['total'] = total_count

In [ ]:
with open(f'{out_dir}/{airport}.json', 'w') as f:
    json.dump(agent_types, f, indent=2)